<a href="https://colab.research.google.com/github/Daivar/PP6_Adult_dataset/blob/main/Adult_dataset_fastai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from fastai.tabular import *

!rm -rf ./sample_data/ ./data/

path = untar_data(URLs.ADULT_SAMPLE)
df = pd.read_csv(path/'adult.csv')

In [ ]:
df

In [ ]:
df.dtypes

In [ ]:
dep_var = 'salary'

cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race']
cont_names = ['age', 'fnlwgt', 'education-num']
procs = [FillMissing, Categorify]

In [ ]:
test = TabularList.from_df(df.iloc[800:1000].copy(), path=path, cat_names=cat_names, cont_names=cont_names)

In [ ]:
test[0]

In [ ]:
data = (TabularList.from_df(df, path=path, cat_names=cat_names, cont_names=cont_names, procs=procs)
                           .split_by_idx(list(range(800,1000)))
                           .label_from_df(cols=dep_var)
                           .add_test(test)
                           .databunch())

In [ ]:
data.show_batch(rows=10)

In [ ]:
learn = tabular_learner(data, emb_drop=0.04, layers=[5, 5, 5], metrics=accuracy)

In [ ]:
learn.fit(5, 1e-2)

In [ ]:
learn.model

In [ ]:
learn.lr_find()

In [ ]:
learn.fit_one_cycle(10)

In [ ]:
learn.fit(100)

In [ ]:
learn.show_results()

In [ ]:
row, clas, probs = learn.predict(df.iloc[0])

In [ ]:
clas, probs